---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [10]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [24]:
def date_sorter():
    sdf=df.str.extract(r'(?P<date>(?P<month>[0]?\d|[1][012])[\.\,-/]+(?P<day>\d{1,2})[\.\,-/]+(?P<year>\d{2,4}))')
    nn_df=sdf[pd.notnull(sdf['date'])]
    li=nn_df.index.values.tolist()
    tempdf=df[~df.index.isin(li)]
    nn_df['ind']=nn_df.index
    
    sdf6=tempdf.str.extract(r'(?P<date>(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[\.\,-/ ]+(?P<day>\d{1,2})[a-z\.\,-/ ]+?(?P<year>\d{2,4}))')
    nn_df6=sdf6[pd.notnull(sdf6['date'])]
    li+=nn_df6.index.values.tolist()
    tempdf6=df[~df.index.isin(li)]
    nn_df6.replace(to_replace={'Jan': 1,'Feb': 2 ,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug': 8,'Sep': 9,'Oct' : 10,'Nov': 11,'Dec' : 12,
                             'January': 1,'February': 2 ,'March':3,'April':4,'May':5,'June':6,'July':7,'August': 8,'September': 9,'October' : 10,
                              'November': 11,'December' : 12}, inplace=True)
    nn_df6['ind']=nn_df6.index
    
    sdf2=tempdf6.str.extract(r'(?P<date>(?P<day>\d{1,2})\s?(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[\.\, ]+?(?P<year>\d{2,4}))')
    nn_df2=sdf2[pd.notnull(sdf2['date'])]
    li+=nn_df2.index.values.tolist()
    tempdf2=df[~df.index.isin(li)]
    nn_df2.replace(to_replace={'Jan': 1,'Feb': 2 ,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug': 8,'Sep': 9,'Oct' : 10,'Nov': 11,'Dec' : 12,
                             'January': 1,'February': 2 ,'March':3,'April':4,'May':5,'June':6,'July':7,'August': 8,'September': 9,'October' : 10,
                              'November': 11,'December' : 12}, inplace=True)
    nn_df2['ind']=nn_df2.index
    
    sdf3=tempdf2.str.extract(r'\D\D(?P<date>(?P<month>(?:[0]?\d|[1][012]|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[\.\,-/]*\s*(?P<year>\d{4,4}))')
    nn_df3=sdf3[pd.notnull(sdf3['date'])]
    nn_df3['day']=1
    li+=nn_df3.index.values.tolist()
    tempdf3=df[~df.index.isin(li)]
    nn_df3.replace(to_replace={'Jan': 1,'Feb': 2 ,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug': 8,'Sep': 9,'Oct' : 10,'Nov': 11,'Dec' : 12,
                             'Janaury' :1,'January': 1,'February': 2 ,'March':3,'April':4,'May':5,'June':6,'July':7,'August': 8,'September': 9,'October' : 10,
                              'November': 11,'December' : 12,'Decemeber' : 12}, inplace=True)
    nn_df3['ind']=nn_df3.index
    
    sdf4=tempdf3.str.extract(r'(?P<date>(?P<month>(?:[0]?\d|[1][012]|Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*)[\.\,-/]*\s*(?P<year>\d{4,4}))')
    nn_df4=sdf4[pd.notnull(sdf4['date'])]
    nn_df4['day']=1
    li+=nn_df4.index.values.tolist()
    tempdf4=df[~df.index.isin(li)]
    nn_df4.replace(to_replace={'Jan': 1,'Feb': 2 ,'Mar':3,'Apr':4,'May':5,'Jun':6,'Jul':7,'Aug': 8,'Sep': 9,'Oct' : 10,'Nov': 11,'Dec' : 12,
                             'January': 1,'February': 2 ,'March':3,'April':4,'May':5,'June':6,'July':7,'August': 8,'September': 9,'October' : 10,
                              'November': 11,'December' : 12}, inplace=True)
    nn_df4['ind']=nn_df4.index
    
    sdf5=tempdf4.str.extract(r'(?P<date>(?P<year>\d{4,4}))')
    nn_df5=sdf5[pd.notnull(sdf5['date'])]
    nn_df5['day']=1
    nn_df5['month']=1
    li+=nn_df5.index.values.tolist()
    nn_df5['ind']=nn_df5.index
    
    mdf=pd.merge(nn_df,nn_df2,how='outer')
    mdf=pd.merge(mdf,nn_df3,how='outer')
    mdf=pd.merge(mdf,nn_df4,how='outer')
    mdf=pd.merge(mdf,nn_df5,how='outer')
    mdf=pd.merge(mdf,nn_df6,how='outer')
    mdf.year=mdf.year.astype(int)
    mdf['year']=mdf['year'].apply(lambda x: 1900+x if (x//1000)==0 else x)
    mdf.month=mdf.month.astype(int)
    mdf.day=mdf.day.astype(int)
#     mdf.year=mdf.year.astype(str)
#     mdf['d']=pd.to_datetime(mdf.year+mdf.month+mdf.day,format='%Y%m%d',errors='coerce')
#     tdf=mdf[pd.isnull(mdf['d'])]
    mdf['d']=pd.to_datetime(mdf[['year','month','day']],errors='coerce')
    mdf.sort_values(by=['d'],ascending=True,inplace=True)
    
#     nn_df5=nn_df5.astype(int) 
#     nn_df5['d']=nn_df5.year+nn_df5.month+nn_df5.day
#     dt=pd.to_datetime(nn_df5.d,format="%Y%m%d")
    s=pd.Series(mdf.ind.values)
    return s#df.tail(50)
date_sorter()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  
/opt/conda/lib/python3.6/site-packages/pandas/core/generic.py:3725: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     323
20     405
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    220
471    208
472    243
473    139
474    320
475    383
476    244
477    480
478    286
479    431
480    279
481    198
482    381
483    463
484    366
485    439
486    255
487    401
488    475
489    257
490    152
491    235
492    464
493    253
494    231
495    427
496    141
497    186
498    161
499    413
Length: 500, dtype: int64